# News similarity modelling

[Dataset](https://www.kaggle.com/c/newssimil)

In [1]:
import pandas as pd
import csv
import numpy as np
import nltk

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/andrgolubev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/andrgolubev/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
train_file = 'news_similarity_train.csv'

In [5]:
news_df = pd.read_csv(train_file, sep=",",
                      engine="c",
#                       names=['post_id','published_at','title','subtitle','content'],
#                       dtype={
#                           'post_id': np.int32,
#                           'published_at': np.datetime64,
#                           'title': str,
#                           'subtitle': str,
#                           'content': str
#                       }
#                       skiprows=skip
                     )

In [6]:
news_df

,post_id,published_at,title,subtitle,content
0,1,2017-01-01T00:05:00Z,Полиция Стамбула собирается штурмовать ночной ...,"Согласно последним данным, в здании может нахо...",<p>Полиция Стамбула собирается штурмовать ночн...
1,2,2017-01-01T00:07:00Z,Расстрел посетителей в ночном клубе в Стамбуле...,"Ранее сообщалось, что полицейские собираются ш...","<p>Опубликовано видео, где злоумышленник расст..."
2,3,2017-01-01T00:21:00Z,Губернатор Стамбула: При нападении на ночной к...,"Ранее сообщалось, что от 40 до 60 человек полу...",<p>Губернатор Стамбула <span>Васип Шахин</span...
3,4,2017-01-01T00:29:00Z,Губернатор Стамбула назвал террористической ат...,"Согласно последним данным, погибло минимум 35 ...",<p>Губернатор Стамбула Васип Шахин назвал терр...
4,5,2017-01-01T00:47:00Z,Очевидцы: В ночной клуб в Стамбуле проходили б...,Ранее губернатор Стамбула Васип Шахин назвал п...,"<p>Посетители стамбульского клуба ""Рейна"", на ..."
5,6,2017-01-01T00:56:00Z,"В турецком клубе, который был атакован террори...",NaN,"<p>Клуб ""Рейна"", на который сегодня было совер..."
6,7,2017-01-01T01:01:00Z,"Опубликовано видео из клуба ""Рейна"" за несколь...","Как ранее сообщалось, в результате атаки погиб...","<p>Опубликовано видео из стамбульского клуба ""..."
7,8,2017-01-01T01:01:00Z,Турецким СМИ запретили писать про теракт в Ста...,NaN,<p>Официальные власти Турции экстренно издали ...
8,9,2017-01-01T01:08:00Z,Губернатор Стамбула рассказал о нападении на к...,NaN,<p><span>По словам губернатора Стамбула Васипа...
9,10,2017-01-01T01:11:00Z,Опубликовано видео начала атаки на ночной клуб...,"На кадрах видно, как один злоумышленник с авто...",<p>В Сети появилось видео начала атаки на ночн...


In [7]:
import json
def get_json(myjson):
  try:
    return json.loads(myjson)
  except ValueError:
    return None

In [8]:
print(get_json(news_df['content'][103325]) is not None)
print(get_json(news_df['content'][0]) is not None)

True
False


In [21]:
j = get_json(news_df['content'][103297])
full_j = [row["value"]["blocks"][0]["text"] for row in j if row["type"] == "TEXT" or row["type"] == "SUBTITLE"]
"\n".join(full_j)

'21-летняя Райли Хемсон два года назад весила 115 кг\nДевушка занялась фитнесом и стала описывать всё, что с ней происходит, в соцсетях\nВо время своей работы парамедиком Хемсон сталкивалась с людьми, страдающими от избытка веса\nРайли делилась с подписчиками в "Инстаграме" не только своими успехами, но и переживаниями, ошибками\nОна начала активно пропагандировать здоровый образ жизни и идею о том, что надо любить своё тело и заботиться о нём на всех этапах жизни\nНе смотря на то что девушка является вегетарианкой, она набрала лишний вес из-за частого употребления пиццы и прочей вредной еды\nРайли вдохновила многих людей своими усилиями вести здоровый образ жизни. За год девушка потеряла 25 кг\nВ результате на страницу девушки в "Инстаграме" подписалось более 15 тыс. человек'

In [20]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(news_df['content'][16], 'html.parser')
soup.get_text().replace(u'\xa0', u' ')

'Президент США Барак Обама, генсек НАТО Йенс Столтенберг и глава европейской дипломатии Федерика Могерини выразили соболезнования в связи с терактом в Стамбуле.\n— Трагическое начало 2017 года в Стамбуле. Мои мысли сейчас с теми, кого затронуло это нападение в то время, когда они праздновали Новый год, — написал Йенс Столтенберг на своей страничке в "Твиттере".\n \nБарак Обама, в свою очередь, распорядился немедленно оказать всю необходимую помощь Турции после случившегося теракта в Стамбуле.\nФедерика Могерини также сказала, что этот год начался с трагических событий в Турции.\n— 2017 год начался с атаки на Стамбул. Наши мысли с погибшими и их родными. Мы продолжим работать над предотвращением таких трагедий, — написала Могерини в своём "Твиттере".\n \nНапомним, что в результате вооружённого нападения на элитный ночной клуб "Рейна" в Стамбуле погибли не менее 35 человек. За последними новостями по этой теме следите в нашей текстовой трансляции.'

In [85]:
news_df['content'][0]

'<p>Полиция Стамбула собирается штурмовать ночной клуб, где произошло нападение. Согласно последним данным, минимум один злоумышленник находится в здании. Ранено, по разным данным, от 40 до 60 человек.</p>\n<p>Жертвами стали не менее четырёх человек. Среди погибших — один полицейский.</p>\n<p>Во время празднеств\xa0в клубе "Рейна" находилось до 800 человек. Неизвестные произвели множество выстрелов и, как сообщается, взорвали две гранаты.</p>'

In [5]:
from pymystem3 import Mystem
m = Mystem()

In [6]:
lemmas = m.lemmatize(news_df['title'][0])

In [7]:
print(lemmas)

['полиция', ' ', 'стамбул', ' ', 'собираться', ' ', 'штурмовать', ' ', 'ночной', ' ', 'клуб', ', ', 'где', ' ', 'происходить', ' ', 'нападение', '\n']


In [8]:
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = set(stopwords.words('russian'))
print(stop_words)

{'они', 'в', 'ней', 'всего', 'из', 'есть', 'куда', 'через', 'два', 'как', 'другой', 'хорошо', 'этом', 'почти', 'моя', 'только', 'совсем', 'над', 'он', 'все', 'где', 'нас', 'впрочем', 'чтобы', 'со', 'лучше', 'к', 'вот', 'хоть', 'нее', 'у', 'чуть', 'много', 'раз', 'тоже', 'чтоб', 'такой', 'уж', 'ее', 'уже', 'всех', 'мне', 'так', 'бы', 'был', 'кто', 'надо', 'один', 'ведь', 'без', 'ну', 'нет', 'зачем', 'сам', 'можно', 'но', 'если', 'же', 'теперь', 'чем', 'иногда', 'сейчас', 'будет', 'там', 'что', 'вам', 'ей', 'ни', 'потом', 'себе', 'от', 'всю', 'на', 'еще', 'даже', 'наконец', 'про', 'было', 'с', 'и', 'его', 'им', 'этой', 'ничего', 'всегда', 'во', 'мы', 'для', 'тем', 'по', 'ж', 'ним', 'то', 'перед', 'а', 'нибудь', 'ему', 'при', 'или', 'не', 'мой', 'были', 'них', 'три', 'ли', 'о', 'потому', 'чего', 'до', 'никогда', 'об', 'более', 'этот', 'ты', 'разве', 'под', 'опять', 'вы', 'я', 'этого', 'какая', 'свою', 'нельзя', 'конечно', 'тогда', 'вдруг', 'тот', 'себя', 'того', 'за', 'какой', 'меня', 'не

In [32]:
import re
def lemmatize_filter(mstem_instance, text, stop_words):
    lemmatized = pd.Series([''.join(mstem_instance.lemmatize(text_row)) for text_row in text])
    filtered = []
    for row in lemmatized:
        filtered.append(' '.join([word for word in re.split(" |, |\n |:", row) if word and word not in stop_words]))
    return pd.Series(filtered)

In [33]:
all_lemmas = lemmatize_filter(m, news_df['title'], stop_words)

In [34]:
all_lemmas

0         полиция стамбул собираться штурмовать ночной к...
1         расстрел посетитель ночной клуб стамбул попада...
2         губернатор стамбул нападение ночной клуб погиб...
3         губернатор стамбул называть террористический а...
4         очевидец ночной клуб стамбул проходить беспреп...
5         турецкий клуб который атаковать террорист серь...
6         опубликовывать видео клуб "рейн" несколько час...
7            турецкий сми запрещать писать теракт стамбул\n
8         губернатор стамбул рассказывать нападение клуб...
9         опубликовывать видео начало атака ночной клуб ...
10        власть стамбул озвучивать официальный версия н...
11            сша небо сталкиваться легкомоторный самолет\n
12        руководство клуб "рейн" предупреждать подготов...
13        экс-голкипер ска бобровский становиться герой ...
14         четыре человек пострадать взрыв петарда москва\n
15        сми среди ранить ночной клуб стамбул иностранец\n
16        обама могерини столтенберг выр

In [36]:
news_df['title'].head()

0    Полиция Стамбула собирается штурмовать ночной ...
1    Расстрел посетителей в ночном клубе в Стамбуле...
2    Губернатор Стамбула: При нападении на ночной к...
3    Губернатор Стамбула назвал террористической ат...
4    Очевидцы: В ночной клуб в Стамбуле проходили б...
Name: title, dtype: object

In [37]:
news_df['title'] = all_lemmas

In [38]:
news_df['title'].head()

0    полиция стамбул собираться штурмовать ночной к...
1    расстрел посетитель ночной клуб стамбул попада...
2    губернатор стамбул нападение ночной клуб погиб...
3    губернатор стамбул называть террористический а...
4    очевидец ночной клуб стамбул проходить беспреп...
Name: title, dtype: object

In [94]:
# json parser
def json_parse(json_object, concat_str="\n"):
    text_from_json = [e["value"]["blocks"][0]["text"] for e in json_object if e["type"] == "TEXT" or e["type"] == "SUBTITLE"]
    return concat_str.join(text_from_json)

In [22]:
# html cleaner
def html_clean(html_string):
    from bs4 import BeautifulSoup
    return BeautifulSoup(html_string, "html.parser").get_text().replace(u'\xa0', u' ')